In [1]:
### Can probably cut these down as we are no longer
### running the training in this notebook

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet.keras as efn 
import librosa
import librosa.display as display
import os
import warnings
import matplotlib
from tqdm import tqdm


from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise
from kapre.time_frequency import Spectrogram

%matplotlib inline

#!rm -r train_data
#!rm -r val_data
#!rm -r models
#!mkdir models

# suppress warnings
warnings.filterwarnings("ignore")

SOUND_DIR = "/project/data/birdsong-recognition/train_audio/"
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

Using TensorFlow backend.


In [2]:
# function for creating Mel Spectrogram
def createMelSpectrogram(input_path, fileName, output_path, saveOrShow=0):
    
    input_file=os.path.join(output_path, fileName)
    output_file=os.path.join(output_path, fileName[:-4] + ".png")
    # print(output_file)
    
    

    if ( not( os.path.exists(output_file)) and not(os.path.isdir(output_file))):
        print(f"{output_file} does not exist, and is not a directory")
        # load sound signal
        signal, sr = librosa.load(os.path.join(input_path, fileName), duration=10)

        # create Mel Spectrogram
        print(f"Creating spectrogram {output_file}")
        S = Melspectrogram(n_dft=1024, 
                           n_hop=256,
                           input_shape=(1, signal.shape[0]),
                           padding='same', sr=sr, n_mels=224, fmin=1400, fmax=sr/2,
                           power_melgram=2.0, return_decibel_melgram=True,
                           trainable_fb=False, trainable_kernel=False)(signal.reshape(1, 1, -1)).numpy()

        S = S.reshape(S.shape[1], S.shape[2])

        if saveOrShow == 0:   
            matplotlib.image.imsave(output_file, S)
        else:
            #plt.imshow(S)
            #plt.show()
            display.specshow(S, sr=sr)
            plt.show()

## Create Mel-Spectrograms and split for training and validation

In [3]:
# select top 20 birds for training
BIRDS = os.listdir("data/birdsong-recognition/train_audio/")[0:20]

In [4]:
print(BIRDS)

['amered', 'yelwar', 'semsan', 'horlar', 'vigswa', 'blugrb1', 'cedwax', 'sposan', 'brthum', 'gockin', 'belkin1', 'olsfly', 'whbnut', 'commer', 'wewpew', 'moudov', 'wilsni1', 'pasfly', 'gryfly', 'annhum']


In [5]:
train_folder = "data/train_data_decibel_20/"
val_folder = "data/val_data_decibel_20/"

if not os.path.exists(train_folder): os.mkdir(train_folder)
if not os.path.exists(val_folder): os.mkdir(val_folder)

In [6]:


# create train and val spectrogram
np.random.seed(1234)
for bird in BIRDS:
    INPUT_DIR = os.path.join("data/birdsong-recognition/train_audio/", bird)
    TRAIN_DIR = os.path.join(train_folder, bird)
    VAL_DIR = os.path.join(val_folder, bird)
    
    # create bird folders if they don't already exist
    if not(os.path.exists(TRAIN_DIR)):
        os.mkdir(TRAIN_DIR)
    
    if not(os.path.exists(VAL_DIR)): 
        os.mkdir(VAL_DIR)

    # to avoid valdiation and training sets having similar recordings, we will
    # separate them based on the core part of the filename: XCnnnnnn. 
    # We keep track of the cores already sent to each of train and validation sets
    
    # First, build the list of core names currently in the validation and training dirs
    # by stripping off the ending.
    
    trn_names = []
    val_names = []
    
   
    for file in [ f for f in os.listdir(TRAIN_DIR) if f[0] == "X" ]:
        trn_names.append(file.split(".")[0])
    
    for file in [ f for f in os.listdir(VAL_DIR) if f[0] == "X" ]:
        val_names.append(file.split(".")[0])
    
    print("trn_names", trn_names)
    print("val_names", val_names)
    
    # cross check that nothing appears in both lists
    for core_name in val_names:
        if core_name in trn_names:
            print(f"core name {core_name} appears in both training and validation sets")

    # process file-by-file
            
    # look at each of the the original files (starting with "X")
    # if they are already assigned, the spectragram of the file will
    # be sent to the same data set.
    
    for f in [ i for i in os.listdir(INPUT_DIR) if i[0] == "X"]:
        core_name = f.split(".")[0]
        # this is an original file. Check if it has already been assigned
        if core_name in trn_names:
            dest_dir = TRAIN_DIR
            print(f"{core_name} already in training dataset")
        elif core_name in val_names:
            print(f"{core_name} already in validation dataset")
            dest_dir = VAL_DIR
        else:
            # this is an original file we haven't assigned to a dataset
            rand = np.random.randint(0, 10)
            if rand <= 7: 
                dest_dir = TRAIN_DIR
                trn_names.append(core_name)
                print(f"Randomly assigning {core_name} to training dataset")
            else:
                dest_dir = VAL_DIR
                val_names.append(core_name)
                print(f"Randomly assigning {core_name} to validation dataset")
        createMelSpectrogram(INPUT_DIR, f, dest_dir)
    

    # Now do the same for the generated files

    for f in [ i for i in os.listdir(INPUT_DIR) if i[0] == "A"]:
        core_name = f.split(".")[1] # discards the leading "A."
        
        # check if it has already been assigned.
        if core_name in trn_names:
            dest_dir = TRAIN_DIR
            print(f"{core_name} already in training dataset")
        elif core_name in val_names:
            dest_dir = VAL_DIR
            print(f"{core_name} already in validation dataset")
        else:
            # THIS SHOULD NOT HAPPEN!!!
            # if we get here, then the original XCnnnnnn.png has
            # gone missing, so print a warning and assign randomly
            rand = np.random.randint(0, 10)
            if rand <= 7: 
                dest_dir = TRAIN_DIR
                trn_names.append(core_name)
                print(f"WARNING: Original spectrogram not found, randomly assigning augmented files for {core_name} to training dataset")

            else:
                dest_dir = VAL_DIR
                val_names.append(core_name)
                print(f"WARNING: Original spectrogram not found, randomly assigning augmented files for {core_name} to validation dataset")
        createMelSpectrogram(INPUT_DIR, f, dest_dir)
            
            

trn_names ['XC135462', 'XC139433', 'XC77881', 'XC315359', 'XC193134', 'XC325810', 'XC361030', 'XC139937', 'XC420882', 'XC469603', 'XC31114', 'XC389300', 'XC183621', 'XC325262', 'XC331421', 'XC186306', 'XC381036', 'XC477187', 'XC332762', 'XC241121', 'XC314923', 'XC179362', 'XC139439', 'XC413723', 'XC473929', 'XC178129', 'XC499324', 'XC167793', 'XC476107', 'XC178310', 'XC321344', 'XC187783', 'XC382119', 'XC333385', 'XC187609', 'XC371041', 'XC187608', 'XC233159', 'XC142592', 'XC135468', 'XC391157', 'XC193769', 'XC179188', 'XC137604', 'XC476576', 'XC318331', 'XC31116', 'XC101593', 'XC135463', 'XC179305', 'XC478597', 'XC179630', 'XC391158', 'XC17100', 'XC188494', 'XC79972', 'XC499322', 'XC308434', 'XC144666', 'XC125512', 'XC388173', 'XC135440', 'XC179661', 'XC272153', 'XC302517', 'XC179185', 'XC293435', 'XC205267', 'XC139880', 'XC193127', 'XC386075', 'XC293615', 'XC195834', 'XC327739', 'XC390592', 'XC133902', 'XC325943', 'XC330861', 'XC188495', 'XC193128']
val_names ['XC134499', 'XC483145',

Creating spectrogram train_data_decibel_20/amered/XC178310.0.png
XC178310 already in training dataset
train_data_decibel_20/amered/XC178310.1.png
train_data_decibel_20/amered/XC178310.1.png does not exist, and is not a directory
Creating spectrogram train_data_decibel_20/amered/XC178310.1.png
XC315359 already in training dataset
train_data_decibel_20/amered/XC315359.png
XC125512 already in training dataset
train_data_decibel_20/amered/XC125512.0.png
train_data_decibel_20/amered/XC125512.0.png does not exist, and is not a directory
Creating spectrogram train_data_decibel_20/amered/XC125512.0.png
XC183621 already in training dataset
train_data_decibel_20/amered/XC183621.png
XC382119 already in training dataset
train_data_decibel_20/amered/XC382119.png
XC389300 already in training dataset
train_data_decibel_20/amered/XC389300.0.png
train_data_decibel_20/amered/XC389300.0.png does not exist, and is not a directory
Creating spectrogram train_data_decibel_20/amered/XC389300.0.png
XC135466 al

KeyboardInterrupt: 

In [ ]:
    print("trn_names", trn_names)
    print("val_names", val_names)
